In [122]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

dt = np.dtype([('instance_no', int),
               ('exp_no', int),
               ('method', int), # 1 = white box, 2 = euclidean_PCA, 3 = hog, 4 = euclidean_PCA category, 5 = hog category, 6 = ais
               ('pca_n', int),
               ('percentage_of_data', float),
               ('percentile', float),
               ('mc_euclidean_no_batches', int), # stuff
               ('mc_hog_no_batches', int), # stuff
               ('sigma_ais', float),
               ('mc_attack_log', float),
               ('mc_attack_eps', float),
               ('mc_attack_frac', float), 
               ('mc_attack_log_50', float), 
               ('mc_attack_eps_50', float),
               ('mc_attack_frac_50', float),
               ('white_box_50', float),
               ('white_box_11', float),
               ('ais_50', float),
               ('ais_acc_rate_50', float),
               ('successful_sum_attack_1', float),
               ('successful_sum_attack_2', float),
               ('successful_sum_attack_3', float),
              ])

In [123]:
data = pd.DataFrame(np.loadtxt('CIFAR10_MC_ATTACK.csv', dtype=dt))


data = data.assign(mem_inf_adv_mc_log = lambda x: (x.mc_attack_log_50-0.5)*2)
data = data.assign(mem_inf_adv_mc_ones = lambda x: (x.mc_attack_eps_50-0.5)*2)
data = data.assign(mem_inf_adv_wb = lambda x: (x.white_box_50-0.5)*2)
data = data.assign(mem_inf_adv_ais = lambda x: (x.ais_50-0.5)*2)

def convert_to_set_prob(x):
    prob = np.zeros((len(x),))
    for i in range(len(x)):
        if x[i] > 0:
            prob[i] = 1
        elif x[i] == 0:
            prob[i] = 0.5
        elif x[i] < 0:
            prob[i] = 0
    return prob

data = data.assign(set_accuracy_mc_log = lambda x: convert_to_set_prob(x.mem_inf_adv_mc_log))
data = data.assign(set_accuracy_mc_ones = lambda x: convert_to_set_prob(x.mem_inf_adv_mc_ones))
data = data.assign(set_accuracy_wb = lambda x: convert_to_set_prob(x.mem_inf_adv_wb))
data = data.assign(set_accuracy_ais = lambda x: convert_to_set_prob(x.mem_inf_adv_ais))

data = data.assign(set_mem_inf_adv_mc_log = lambda x: 2*x.set_accuracy_mc_log-1)
data = data.assign(set_mem_inf_adv_mc_ones = lambda x: 2*x.set_accuracy_mc_ones-1)
data = data.assign(set_mem_inf_adv_wb = lambda x: 2*x.set_accuracy_wb-1)
data = data.assign(set_mem_inf_adv_ais = lambda x: 2*x.set_accuracy_ais-1)

white_box = data[data.method == 1]
pca = data[data.method == 2]
hog = data[data.method == 3]
pca_cat = data[data.method == 4]
hog_cat = data[data.method == 5]
ais = data[data.method == 6]
color_hist = data[data.method == 8]



# White Box

In [124]:
len(white_box)

40

In [166]:
print(str(np.round(100*white_box.white_box_50.mean(),2))+'$\pm$'+str(np.round(100*white_box.white_box_50.std()/np.sqrt(white_box.white_box_50.count()),2)))
print(str(np.round(100*white_box.set_accuracy_wb.mean(),2))+'$\pm$'+str(np.round(100*white_box.set_accuracy_wb.std()/np.sqrt(white_box.set_accuracy_wb.count()),2)))

97.6$\pm$0.59
100.0$\pm$0.0


# AIS

In [126]:
print(ais.mem_inf_adv_ais.mean())
print(ais.set_accuracy_ais.mean())

nan
nan


# PCA
0.1 mc_attack_log_50

In [127]:
pca.pca_n.unique()

array([120], dtype=int64)

In [128]:
pca_opt_dim = pca[pca.pca_n == 120]

opt_perc = -1

print(pca_opt_dim[pca_opt_dim.percentile == opt_perc].set_accuracy_mc_log.mean())
print(pca_opt_dim[pca_opt_dim.percentile == opt_perc].set_accuracy_mc_ones.mean())

0.65
0.65


In [139]:
100*pca_opt_dim.groupby(['percentile']).mean()[['set_accuracy_mc_ones','set_accuracy_mc_log','successful_sum_attack_1','successful_sum_attack_2']]

,set_accuracy_mc_ones,set_accuracy_mc_log,successful_sum_attack_1,successful_sum_attack_2
percentile,,,,
-1.000,65.00,65.00,55.0,50.0
0.001,48.75,46.25,45.0,45.0
0.010,55.00,65.00,50.0,52.5
0.100,63.75,65.00,50.0,50.0
1.000,63.75,65.00,55.0,60.0


In [140]:
np.round(pca_opt_dim.groupby(['percentile']).std()[['set_accuracy_mc_ones','set_accuracy_mc_log','successful_sum_attack_1','successful_sum_attack_2']]*100
/np.sqrt(pca_opt_dim.groupby(['percentile']).count()[['set_accuracy_mc_ones','set_accuracy_mc_log','successful_sum_attack_1','successful_sum_attack_2']]),2)

,set_accuracy_mc_ones,set_accuracy_mc_log,successful_sum_attack_1,successful_sum_attack_2
percentile,,,,
-1.000,7.21,7.21,7.97,8.01
0.001,7.70,7.68,7.97,7.97
0.010,7.34,6.98,8.01,8.00
0.100,6.93,6.75,8.01,8.01
1.000,7.16,7.21,7.97,7.84


In [142]:
np.round(100*pca_opt_dim.groupby(['percentile']).mean()[['mc_attack_log_50','mc_attack_eps_50']],2)

,mc_attack_log_50,mc_attack_eps_50
percentile,,
-1.000,51.28,51.28
0.001,49.40,49.70
0.010,50.95,50.23
0.100,50.70,51.30
1.000,51.15,51.12


In [143]:
np.round(pca_opt_dim.groupby(['percentile']).std()[['mc_attack_log_50','mc_attack_eps_50','successful_sum_attack_1','successful_sum_attack_2']]*100
/np.sqrt(pca_opt_dim.groupby(['percentile']).count()[['mc_attack_log_50','mc_attack_eps_50','successful_sum_attack_1','successful_sum_attack_2']]),2)

,mc_attack_log_50,mc_attack_eps_50,successful_sum_attack_1,successful_sum_attack_2
percentile,,,,
-1.000,0.57,0.57,7.97,8.01
0.001,0.54,0.69,7.97,7.97
0.010,0.53,0.53,8.01,8.00
0.100,0.44,0.48,8.01,8.01
1.000,0.52,0.55,7.97,7.84


# Color Histogram
0.1 mc_attack_log_50

In [144]:
color_hist

opt_perc = -1

print(color_hist[color_hist.percentile == opt_perc].set_accuracy_mc_log.mean())
print(color_hist[color_hist.percentile == opt_perc].set_accuracy_mc_ones.mean())
print(pca_opt_dim[pca_opt_dim.percentile == opt_perc].successful_sum_attack_1.mean())
print(pca_opt_dim[pca_opt_dim.percentile == opt_perc].successful_sum_attack_2.mean())

0.5125
0.5125
0.55
0.5


In [145]:
np.round(100*color_hist.groupby(['percentile']).mean()[['set_accuracy_mc_ones','set_accuracy_mc_log','successful_sum_attack_1','successful_sum_attack_2']],2)

,set_accuracy_mc_ones,set_accuracy_mc_log,successful_sum_attack_1,successful_sum_attack_2
percentile,,,,
-1.000,51.25,51.25,47.5,42.5
0.001,42.50,51.25,65.0,60.0
0.010,53.75,43.75,55.0,55.0
0.100,56.25,52.50,50.0,50.0
1.000,42.50,46.25,45.0,50.0


In [146]:
np.round(color_hist.groupby(['percentile']).std()[['set_accuracy_mc_ones','set_accuracy_mc_log','successful_sum_attack_1','successful_sum_attack_2']]*100
/np.sqrt(color_hist.groupby(['percentile']).count()[['set_accuracy_mc_ones','set_accuracy_mc_log','successful_sum_attack_1','successful_sum_attack_2']]),2)

,set_accuracy_mc_ones,set_accuracy_mc_log,successful_sum_attack_1,successful_sum_attack_2
percentile,,,,
-1.000,7.49,7.49,8.00,7.92
0.001,7.50,7.70,7.64,7.84
0.010,6.79,7.64,7.97,7.97
0.100,7.64,7.15,8.01,8.01
1.000,7.50,7.68,7.97,8.01


In [148]:
np.round(100*color_hist.groupby(['percentile']).mean()[['mc_attack_log_50','mc_attack_eps_50']],2)

,mc_attack_log_50,mc_attack_eps_50
percentile,,
-1.000,49.45,49.45
0.001,50.67,49.35
0.010,49.80,50.42
0.100,49.80,49.85
1.000,49.45,49.30


In [152]:
np.round(100*color_hist.groupby(['percentile']).std()[['mc_attack_log_50','set_accuracy_mc_log','successful_sum_attack_1','successful_sum_attack_2']]/np.sqrt(color_hist.groupby(['percentile']).count()[['mc_attack_log_50','set_accuracy_mc_log','successful_sum_attack_1','successful_sum_attack_2']]),2)

,mc_attack_log_50,set_accuracy_mc_log,successful_sum_attack_1,successful_sum_attack_2
percentile,,,,
-1.000,0.60,7.49,8.00,7.92
0.001,0.60,7.70,7.64,7.84
0.010,0.65,7.64,7.97,7.97
0.100,0.63,7.15,8.01,8.01
1.000,0.57,7.68,7.97,8.01
